# Lyric classifaction
- transform lyric to artist

In [ ]:
!pip install langdetect

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
import nltk
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Prepare Dataset

In [ ]:
!git clone https://github.com/BFF-Pattern/dataset.git

fatal: destination path 'dataset' already exists and is not an empty directory.


In [ ]:
!mv dataset/engsongs_full/ .

mv: cannot stat 'dataset/engsongs_full/': No such file or directory


In [ ]:
import glob, os
from os import walk

li = []
for dirpath, dirnames, filenames in walk('engsongs_full'):
  break
for fn in filenames:
  df = pd.read_csv(f'{str(dirpath)}/{fn}', index_col=False, header=0)
  li.append(df)

song_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
song_df

,artist,song,link,text
0,KK,Aankhon Mein Teri,/k/kk/aankhon+mein+teri_20664230.html,"Aankhon mein teri...\nAjab si, ajab si, adaaye..."
1,KK,Ajab Si,/k/kk/ajab+si_21257795.html,( Aankhon Mein Teri\nAjab Si Ajab Si Adayein H...
2,KK,All I Want For Christmas Is You,/k/kk/all+i+want+for+christmas+is+you_21257963...,I don't want a lot for Christmas\nThere's just...
3,KK,Allah Hafiz,/k/kk/allah+hafiz_21256895.html,O Raahi Chal\nO Raahi Chal\n\nO Raahi Chal\nO ...
4,KK,Chale Jaise Hawaien,/k/kk/chale+jaise+hawaien_21256899.html,"Chalein Jaise Hawayein Sanan Sanan, Udein Jais..."
...,...,...,...,...
74777,Yadira Coradin,Mi Alma Te Bendice,/y/yadira+coradin/mi+alma+te+bendice_20992249....,"Mi alma, te bendice oh Dios\nPorque eres tan b..."
74778,Yadira Coradin,No Engañamos Al Señor,/y/yadira+coradin/no+engaamos+al+seor_20992257...,"No, no engañamos al Señor\nAl decir que venimo..."
74779,Yadira Coradin,No Llores Mas,/y/yadira+coradin/no+llores+mas_20992250.html,Lloras solo y tu dolor\nNo hay respuesta en de...
74780,Yadira Coradin,Pensando En Ti,/y/yadira+coradin/pensando+en+ti_20992254.html,Yo vivo pensando en ti\nTu tienes morada en mi...


In [ ]:
print(song_df['text'].iloc[200])
err_msg = song_df['text'].iloc[200]
song_df = song_df[song_df['text'] != err_msg]

[Teyana Taylor - Chorus]
Christmas in Harlem
Right after autumn falls
Soaking it all in
Then we go hit the mall
Even though we ain't balling
Feels like we bought it all
The mistletoe's right here
Come give a kiss to Santa Claus

[Kanye West - Verse 1]
Merry Christmas to all, and all a good night
Huh, now we all living the good life
Yeah, though it's forty below the wind chill
And we wiping snow up off the windshield
It's still, wonderful night to be alive, baby
And I'm, so happy I'm with my baby
And we a lil late with the Christmas gifts
Rushing for the mall, don't trip, you know I drive crazy
The streets lit up, it feel like Christmas officially
Told her that "You the star at the top of my Christmas tree"
My only question is, "Where my presents?"
She said, "Shhh," she got a gift for me that ain't for the kids to see
Well, I like the way you think, mami
Now pour some more eggnog in you drink, mami
You been a bad girl, give santa three kisses
Gave her the hot chocolate, she said "It's D

##Remove bracket

In [ ]:
song_df['text'] = song_df['text'].apply(str)
text_in_round_brackets = sum(list(song_df['text'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
print('Number of round brackets: {}'.format(len(text_in_round_brackets)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Number of round brackets: 147834


In [ ]:
text_in_square_brackets = sum(list(song_df['text'].map(lambda s: re.findall(r'\[(.*?)\]',s))), [])
print('Number of square brackets: {}'.format(len(text_in_square_brackets)))

Number of square brackets: 100959


In [ ]:
text_in_curly_brackets = sum(list(song_df['text'].map(lambda s: re.findall(r'\{(.*?)\}',s))), [])
print('Number of square brackets: {}'.format(len(text_in_curly_brackets)))

Number of square brackets: 2022


In [ ]:
# remove round brackets but not text within
song_df['text'] = song_df['text'].map(lambda s: re.sub(r'\(|\)', '', s))

# remove square brackest and text within
song_df['text'] = song_df['text'].map(lambda s: re.sub(r'\[(.*?)\] ', '', s))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


## Count number of lines

In [ ]:
# count number of lines
song_df['lines'] = song_df['text'].map(lambda t: len(re.findall(r'\n', t)))
# remove line breaks
song_df['text'] = song_df['text'].map(lambda s: re.sub(r' \n|\n', '', s))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


## Drop missing rows

In [ ]:
missing_cond = (song_df['text'] == 'nan') | (song_df['lines'] == 0)
song_df[missing_cond]

,artist,song,link,text,lines
38,Kumar Sanu,Dil Ai Dil,/k/kumar+sanu/dil+ai+dil_21232925.html,Maine Puchha Mere Dil Se Deewane Mujhe Bata Tu...,0
939,Kris Kristofferson,Willie Nelson Sings Kristofferson,/w/willie+nelson/album/willie_nelson_sings_kri...,nan,0
940,Kris Kristofferson,New Sunrise,/b/brenda+lee/album/new_sunrise+191799,nan,0
941,Kris Kristofferson,John Prine,/j/john+prine/album/john_prine+6155,nan,0
942,Kris Kristofferson,Sit Down Young Stranger,/g/gordon+lightfoot/album/sit_down_young_stran...,nan,0
...,...,...,...,...,...
74187,Yoko Ono,Menlove Ave.,/j/john+lennon/album/menlove_ave_+27923,nan,0
74188,Yoko Ono,Some Time In New York City,/j/john+lennon/album/some_time_in_new_york_cit...,nan,0
74189,Yoko Ono,Imagine,/j/john+lennon/album/imagine+5300,nan,0
74190,Yoko Ono,John Lennon/Plastic Ono Band,/j/john+lennon/album/john_lennonplastic_ono_ba...,nan,0


In [ ]:
song_df = song_df[~missing_cond]
song_df[missing_cond]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,artist,song,link,text,lines


In [ ]:
song_df.head()

,artist,song,link,text,lines
0,KK,Aankhon Mein Teri,/k/kk/aankhon+mein+teri_20664230.html,"Aankhon mein teri...Ajab si, ajab si, adaayein...",48
1,KK,Ajab Si,/k/kk/ajab+si_21257795.html,Aankhon Mein TeriAjab Si Ajab Si Adayein HaiH...,42
2,KK,All I Want For Christmas Is You,/k/kk/all+i+want+for+christmas+is+you_21257963...,I don't want a lot for ChristmasThere's just o...,57
3,KK,Allah Hafiz,/k/kk/allah+hafiz_21256895.html,O Raahi ChalO Raahi ChalO Raahi ChalO Raahi Ch...,33
4,KK,Chale Jaise Hawaien,/k/kk/chale+jaise+hawaien_21256899.html,"Chalein Jaise Hawayein Sanan Sanan, Udein Jais...",28


## Drop non-english songs

In [ ]:
def get_eng_prob(text):
    detections = detect_langs(text)
    for detection in detections:
        if detection.lang == 'en':
            return detection.prob
    return 0

song_df['en_prob'] = song_df['text'].map(get_eng_prob)

print('Number of english songs: {}'.format(sum(song_df['en_prob'] >= 0.5)))
print('Number of non-english songs: {}'.format(sum(song_df['en_prob'] < 0.5)))

Number of english songs: 69839
Number of non-english songs: 4941


In [ ]:
song_df = song_df.loc[song_df['en_prob'] >= 0.5]

# Feature Engineering

## Using tokenize and stemmer

In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
song_df['tokens'] = song_df['text'].map(tokenizer.tokenize)

print('Text:')
print(song_df['text'].iloc[0])

print('Tokens:')
print(song_df['tokens'].iloc[0])

Text:
I don't want a lot for ChristmasThere's just one thing I needI don't care about the presentsUnderneath the Christmas treeI just want you for my ownMore than you could ever knowMake my wish come trueAll I want for ChristmasIs youI don't want a lot for ChristmasThere is just one thing I needI don't care about the presentsUnderneath the Christmas treeI don't need to hang my stockingThere upon the fireplaceSanta Claus won't make me happyWith a toy on Christmas dayI just want you for my ownMore than you could ever knowMake my wish come trueAll I want for Christmas is youYou babyI won't ask for much this ChristmasI won't even wish for snowI'm just gonna keep on waitingUnderneath the mistletoeI won't make a list and send itTo the North Pole for Saint NickI won't even stay awake toHear those magic reindeer click'Cause I just want you here tonightHolding on to me so tightWhat more can I doBaby all I want for Christmas is youYouAll the lights are shiningSo brightly everywhereAnd the sound 

In [ ]:
# initialise stemmer
stemmer = nltk.stem.porter.PorterStemmer()

token = 'make'
print('{} -> {}'.format(token, stemmer.stem(token)))

token = 'makes'
print('{} -> {}'.format(token, stemmer.stem(token)))

token = 'making'
print('{} -> {}'.format(token, stemmer.stem(token)))

token = 'made'
print('{} -> {}'.format(token, stemmer.stem(token)))

make -> make
makes -> make
making -> make
made -> made


In [ ]:
# create dictionary to map tokens their stem
token_to_stem = {}
# initialise word count
token_count = 0
# iterate through all songs
for lst in song_df['tokens']:
    # iterate through all tokens of song
    for token in lst:
        token_count += 1
        # check if token is in dictionary
        if token not in token_to_stem:
            # add token to dictionary
            token_to_stem[token] = stemmer.stem(token)
            
song_df['stems'] = song_df['tokens'].map(lambda lst: [token_to_stem[token] for token in lst])

print('Number of tokens: {}'.format(token_count))
print('Number of unique tokens: {}'.format(len(token_to_stem.keys())))
print('Number of unique stems: {}'.format(len(set(token_to_stem.values()))))

Number of tokens: 17333652
Number of unique tokens: 764890
Number of unique stems: 659625


In [ ]:
# number of songs
print('number of songs: ', str(len(song_df)))

# number of artists
print('number of artists: ', str(len(song_df['artist'].unique())))

# distribution songs per artist
song_count_df = song_df.groupby('artist')[['song']].count()
fig = px.histogram(song_count_df, x='song', title='Songs per artist', labels={'song': 'Songs'})
fig.show()

number of songs:  69839
number of artists:  739


### Feature: n_stems

In [ ]:
# words per song
song_df['n_stems'] = song_df['stems'].map(len)

fig = px.histogram(song_df, x='n_stems', title='Words per song')
fig.show()

### Drop duplicate songs

In [ ]:
# create dataframe with lists of artists
song_df['stems_str'] = song_df['stems'].map(lambda lst: ' '.join(lst))

# map text to artists
stems_to_artist = {}
for tp in song_df[['artist', 'stems_str']].itertuples(index=False):
    artist = tp[0]
    stems = tp[1]
    if stems in stems_to_artist:
        stems_to_artist[stems].append(artist)
    else:
        stems_to_artist[stems] = [artist]

# insert list of artists to dataframe
song_df['artists'] = song_df['stems_str'].map(stems_to_artist)
song_df['duplicates'] = song_df['artists'].map(len) - 1

# convert list of artists to set of artists
song_df['artists'] = song_df['artists'].map(set)
song_df['n_artists'] = song_df['artists'].map(len)

# remove duplicate songs
artist_text_df = song_df.drop_duplicates('stems_str')

In [ ]:
# number of unique songs
print('Number of unique lyrics: {}'.format(sum(artist_text_df['duplicates'] == 0)))
# number of duplicate songs
print('Number of duplicate lyrics: {}'.format(sum(artist_text_df['duplicates'] > 0) + \
                                              sum(artist_text_df['duplicates'])))
# number of duplicates from same artist
print('Number of duplicate lyrics from same artist: {}'.format(sum(artist_text_df['duplicates'] + 1 - \
                                                                   artist_text_df['n_artists'])))
# number of duplicates from different artists
print('Number of duplicate lyrics from different artists: {}'.format(sum(artist_text_df['n_artists']\
                                                                         .loc[artist_text_df['duplicates'] > 0])))

Number of unique lyrics: 67553
Number of duplicate lyrics: 2286
Number of duplicate lyrics from same artist: 160
Number of duplicate lyrics from different artists: 2126


In [ ]:
# randomly select artists
n_artist = 10
random.seed(0)

artist_select = random.choices(song_df['artist'].unique(), k=n_artist)

song_filter_df = song_df.loc[song_df['artist'].isin(artist_select)]
print('Total number of songs: {}'.format(len(song_filter_df)))
song_filter_df.groupby('artist')[['song']].count().reset_index().rename(columns={'song':'songs'})

Total number of songs: 1296


,artist,songs
0,Adam Sandler,102
1,Crush,37
2,Engelbert Humperdinck,171
3,Everclear,152
4,Lana Del Rey,203
5,Neil Diamond,219
6,Queensryche,121
7,Selena,50
8,Train,141
9,Zac Brown Band,100


In [ ]:
fig = px.box(song_filter_df, x='artist', y='n_stems', title='Word count per song by artist')
fig.show()

### Feature: unique_stems_ratio

In [ ]:
# number of unique stems
song_df['n_unique_stems'] = song_df['stems'].map(lambda lst: len(set(lst)))
# ratio of unique stems
song_df['unique_stems_ratio'] = song_df['n_unique_stems'] / song_df['n_stems']

# attach column to selected artists
song_filter_df = song_filter_df.join(song_df['unique_stems_ratio'])

In [ ]:
fig = px.box(song_filter_df, x='artist', y='unique_stems_ratio', title='Ratio of unique words to all words')
fig.show()

### Feature: stems_per_line

In [ ]:
# calculate number of words per line
song_df['stems_per_line'] = song_df['n_stems'] / song_df['lines'].astype(float)

song_filter_df = song_filter_df.join(song_df[['stems_per_line']])

In [ ]:
fig = px.box(song_filter_df, x='artist', y='stems_per_line', title='Words per line')
fig.show()

## TF-IDF score

### Fit and tranform stem into tf-idf scores

In [ ]:
# initialise count vectorizer
cv = CountVectorizer()

# generate word counts
stem_count_vector = cv.fit_transform(song_df['stems_str'])

# compute idf
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(stem_count_vector)

TfidfTransformer()

In [ ]:
# print idf values
tfidf_df = pd.DataFrame({'stem': cv.get_feature_names(), 'weight': tfidf_transformer.idf_})
 
# get lowest weights
tfidf_df.sort_values('weight').head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,stem,weight
566570,the,1.062768
580330,to,1.121010
650684,you,1.193761
22523,and,1.219339
285958,in,1.279885


In [ ]:
# get highest weights
tfidf_df.sort_values('weight', ascending=False).head()

,stem,weight
658619,ﬁreever,11.460815
356311,meetya,11.460815
356326,meeveron,11.460815
356325,meeverlast,11.460815
592877,tripsmad,11.460815


### Feature: tf_idf_vector, tff_idf_score

In [ ]:
# assign tf idf scores to each song
tf_idf_vector = tfidf_transformer.transform(stem_count_vector)

# attach count vectors to dataframe
tf_idf_vector_lst = [-1] * len(song_df)
for i in range(len(song_df)):
    tf_idf_vector_lst[i] = tf_idf_vector[i]
song_df['tf_idf_vector'] = tf_idf_vector_lst    

song_df['tf_idf_score'] = song_df['tf_idf_vector'].map(lambda vec: np.sum(vec.todense()))

# join valus to selected artists
song_filter_df = song_filter_df.join(song_df[['tf_idf_vector', 'tf_idf_score']])

In [ ]:
fig = px.box(song_filter_df, x='artist', y='tf_idf_score', title='TFIDF scores of songs per artist')
fig.show()

### Calculate similarity scores by tf-idf scores

In [ ]:
# caclculate mean vector
def get_mean_vector(vec_lst):
    return csr_matrix(vstack(vec_lst).mean(axis=0))

In [ ]:
# calculate mean vector over all songs of same artist
artist_df = song_df.groupby('artist').agg({'tf_idf_vector': get_mean_vector, 'song': len}).reset_index()\
                   .rename(columns={'song': 'songs'})

# get selected artists
artist_filter_df = artist_df.loc[artist_df['artist'].isin(song_filter_df['artist'])]

In [ ]:
similarity_matrix = cosine_similarity(vstack(artist_filter_df['tf_idf_vector']), 
                                      vstack(artist_filter_df['tf_idf_vector']))
artist_names = artist_filter_df['artist'].tolist()
fig = go.Figure(data=go.Heatmap(z=np.flipud(similarity_matrix), x=artist_names, y=list(reversed(artist_names)), 
                                colorscale='balance', zmin=0.5, zmax=1.1))
fig.show()

In [ ]:
artist_song_filter_df = pd.merge(artist_filter_df[['artist', 'tf_idf_vector', 'songs']].assign(key = 0), 
                                 song_filter_df[['artist', 'tf_idf_vector', 'song']].assign(key = 0), on='key', 
                                 suffixes=['_artist', '_song']).drop('key', axis=1).reset_index(drop=True)
artist_song_filter_df['same_artist'] = artist_song_filter_df['artist_artist'] == artist_song_filter_df['artist_song']

### Visualize similarity of each song

In [ ]:
# calculate similarity of artist tf idf vector and song vector
def tf_idf_vector_similarity(artist_vector, song_vector, songs, same_artist):
    # check if song is from same artist
    if same_artist:
        # deduct song vector from artist vector
        artist_vector = (songs * artist_vector - song_vector) / (songs - 1)
    # calculate similarity
    return cosine_similarity(artist_vector, song_vector)[0][0]

In [ ]:
artist_song_filter_df['vector_similarity'] = \
    artist_song_filter_df.apply(lambda row: tf_idf_vector_similarity(row['tf_idf_vector_artist'], 
                                                                     row['tf_idf_vector_song'], 
                                                                     row['songs'], row['same_artist']), axis=1)

In [ ]:
df = artist_song_filter_df

fig = go.Figure()

fig.add_trace(go.Violin(x=df['artist_artist'][df['same_artist']],
                        y=df['vector_similarity'][df['same_artist']],
                        legendgroup='Same Artist', scalegroup='Same Artist', name='Same Artist',
                        side='negative')
             )
fig.add_trace(go.Violin(x=df['artist_artist'][~df['same_artist']],
                        y=df['vector_similarity'][~df['same_artist']],
                        legendgroup='Different Artists', scalegroup='Different Artists', name='Different Artists',
                        side='positive')
             )

fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0, violinmode='overlay')
fig.update_layout(title='Similarity of Songs')
fig.update_xaxes(range=[-0.5, 9.5])
fig.update_yaxes(range=[-0.1, 0.8], title='Similarity')
fig.show()

### Feature: Polarity

In [ ]:
polarity_lst = [-1] * len(song_df)
subjectivity_lst = [-1] * len(song_df)
for i, text in enumerate(song_df['text']):
    sentiment = TextBlob(text)
    polarity_lst[i] = sentiment.polarity
    subjectivity_lst[i] = sentiment.subjectivity
    
song_df['polarity'] = polarity_lst
song_df['subjectivity'] = subjectivity_lst

song_filter_df = song_filter_df.join(song_df[['polarity', 'subjectivity']])

In [ ]:
fig = px.scatter(song_filter_df, x='polarity', y='subjectivity', color='artist', hover_data=['song'], 
                 title='Polarity and Subjectivity of Songs')
fig.show()

In [ ]:
fig = px.box(song_filter_df, x='artist', y='polarity', title='Polarity by artist')
fig.show()

## Wrap everything up
Adding feature: **mean**, **std**, **diff** and **diff by std** of each features and **cosine similarity** of singer-song

In [ ]:
def select_artist_song_create_feature(song_df, n_set, n_artist, n_song_min, n_song_artist_max):
    song_count_df = song_df.groupby('artist')[['artist']].count().rename(columns={'artist': 'count'})
    artist_lst = list(song_count_df.loc[song_count_df['count'] >= n_song_min].index.values)

    n_set_total = sum(n_set.values())

    artist_set = []
    while len(artist_set) < n_set_total:
        new_artist = tuple(np.random.choice(artist_lst, size=n_artist, replace=False))
        if new_artist not in artist_set:
            artist_set.append(new_artist)

    # split artist sets
    artist_select = {}
    for field, n in n_set.items():
        i_select = np.random.choice(range(len(artist_set)), size=n, replace=False)
        artist_list = list(artist_set)
        artist_select[field] = [artist_list[i] for i in i_select]
        artist_set = [s for s in artist_set if s not in artist_select[field]]

    # create dataframe with all features
    feature_dict = {}
    # dictionary to map artist set id to list of artists
    set_id_to_artist_tp = {}

    i = 0
    for field, artist_set in artist_select.items():
        df_lst = []
        for artist_tp in artist_set:
            i += 1
            df = song_df.loc[song_df['artist'].isin(artist_tp), 
                             ['artist', 'song', 'n_stems', 'unique_stems_ratio', 'stems_per_line', 'tf_idf_vector', 
                              'tf_idf_score', 'polarity']]
            # check if number of songs is too high
            if len(df) * n_artist > n_song_artist_max:
                df = df.sample(int(n_song_artist_max / n_artist), random_state=0)
                
            df['artist_set_id'] = i
            set_id_to_artist_tp[i] = artist_tp
            df_lst.append(df)
        feature_dict[field] = pd.concat(df_lst)  
        print('Number of songs in {}: {}'.format(field, len(feature_dict[field])))

    # get all selected artists
    artist_select_set = set.union(*[set(sum(tp_lst, ())) for tp_lst in artist_select.values()])

    # create artist dataframe from training data
    df_lst = []
    for artist, df in song_df.loc[song_df['artist'].isin(artist_select_set)].groupby('artist'):
        dic = {'artist': artist}
        # calculate averages and standard diviations
        for field in ['n_stems', 'unique_stems_ratio', 'stems_per_line', 'tf_idf_score', 'polarity']:
            dic[field + '_mean'] = df[field].mean()
            dic[field + '_std'] = df[field].std()

        # number of songs
        dic['songs'] = len(df)

        # calculate average tf idf vector
        dic['tf_idf_vector_mean'] = get_mean_vector(df['tf_idf_vector'])

        df_lst.append(pd.DataFrame(dic, index=[0]))
    artist_feature_df = pd.concat(df_lst)

    def get_features(df):
        # get artist set id
        artist_set_id = df['artist_set_id'].iloc[0]
        
        # get all artists
        artist_feature_select_df = artist_feature_df.loc[artist_feature_df['artist']\
                                                         .isin(set_id_to_artist_tp[artist_set_id])]

        # merge dataframes
        artist_song_feature_df = pd.merge(artist_feature_select_df.assign(key=0), df.assign(key=0), on='key', 
                                          suffixes=['_artist', '_song']).drop('key', axis=1)    
        artist_song_feature_df['same_artist'] = \
            artist_song_feature_df['artist_artist'] == artist_song_feature_df['artist_song']

        # calculate features
        for feature in ['n_stems', 'unique_stems_ratio', 'stems_per_line', 'tf_idf_score', 'polarity']:
            artist_song_feature_df[feature + '_diff'] = \
                artist_song_feature_df[feature] - artist_song_feature_df[feature + '_mean']
            artist_song_feature_df[feature + '_diff_std'] = \
                artist_song_feature_df[feature + '_diff'] / artist_song_feature_df[feature + '_std']

        # calculate vector similarity between artist and song
        artist_song_feature_df['vector_similarity'] = \
            artist_song_feature_df.apply(lambda row: tf_idf_vector_similarity(row['tf_idf_vector_mean'], 
                                                      row['tf_idf_vector'], row['songs'], row['same_artist']), 
                                         axis=1)    
        return artist_song_feature_df

    artist_song_feature = {}
    for field in feature_dict:
        artist_song_feature[field] = feature_dict[field].groupby('artist_set_id').apply(get_features)\
                                                        .reset_index(drop=True)
        
    return artist_song_feature

# Training setup

## small parameter setting

In [87]:
# parameter
# number of sets
n_set = {'train': 20, 'val': 20}
# number of artists per set
n_artist = 3
# minimum number of songs of one artist
n_song_min = 5
# maximum number of song - artist pairs per artist set
n_song_artist_max = 100

In [ ]:
np.random.seed(0)
artist_song_feature = select_artist_song_create_feature(song_df, n_set, n_artist, n_song_min, n_song_artist_max)

Number of songs in train: 660
Number of songs in val: 660


In [ ]:
artist_song_feature['train'].iloc[0]

artist_artist                                                          Kate Bush
n_stems_mean                                                          201.758065
n_stems_std                                                            90.675725
unique_stems_ratio_mean                                                 0.491887
unique_stems_ratio_std                                                    0.1418
stems_per_line_mean                                                     5.112976
stems_per_line_std                                                      1.540279
tf_idf_score_mean                                                       6.851225
tf_idf_score_std                                                         1.68851
polarity_mean                                                           0.109068
polarity_std                                                            0.228476
songs                                                                        186
tf_idf_vector_mean          

In [ ]:
feature = ['n_stems_diff', 'n_stems_diff_std',
       'unique_stems_ratio_diff', 'unique_stems_ratio_diff_std',
       'stems_per_line_diff', 'stems_per_line_diff_std', 'tf_idf_score_diff',
       'tf_idf_score_diff_std', 'polarity_diff', 'polarity_diff_std',
       'vector_similarity']
df_lst = []
for f in feature:
    df = artist_song_feature['train'][['same_artist']]
    df['feature'] = f
    df['value'] = artist_song_feature['train'][f]
    df_lst.append(df)
feature_df = pd.concat(df_lst)
feature_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,same_artist,feature,value
0,False,n_stems_diff,-158.758065
1,True,n_stems_diff,49.241935
2,False,n_stems_diff,-26.758065
3,False,n_stems_diff,40.241935
4,True,n_stems_diff,7.241935


## Visualize

In [ ]:
def violine_feature_plot(feature_df, feature_select):

    fig = go.Figure()
    df = feature_df.loc[feature_df['feature'].isin(feature_select)]

    fig.add_trace(go.Violin(x=df['feature'][df['same_artist']],
                            y=df['value'][df['same_artist']],
                            legendgroup='Same Artist', scalegroup='Same Artist', name='Same Artist',
                            side='negative')
                 )
    fig.add_trace(go.Violin(x=df['feature'][~df['same_artist']],
                            y=df['value'][~df['same_artist']],
                            legendgroup='Different Artists', scalegroup='Different Artists', name='Different Artists',
                            side='positive')
                 )

    fig.update_traces(meanline_visible=True)
    fig.update_layout(violingap=0, violinmode='overlay')
    fig.update_layout(title='Feature Comparison')
    fig.update_xaxes(title='Feature')
    return fig


In [ ]:
fig = violine_feature_plot(feature_df, ['n_stems_diff_std', 'unique_stems_ratio_diff_std', 'stems_per_line_diff_std', 
                                        'tf_idf_score_diff_std', 'polarity_diff_std'])
fig.update_xaxes(range=[-0.5, 4.5])
fig.show()

In [ ]:
fig = violine_feature_plot(feature_df, ['vector_similarity'])
fig.update_xaxes(range=[-1, 1])
fig.show()


## X, y prepare and train

In [ ]:
def prepare_data(df, feature_org, feature_abs):
    for f in feature_abs:
        df[f] = df[f].abs()
    X = df[feature_org + feature_abs].values
    y = df['same_artist'].values
    
    return X, y

def select_songs_train_pipeline(song_df, n_set, n_artist, n_song_min, n_song_artist_max, feature_org, feature_abs, 
                                pipeline):
    artist_song_feature = select_artist_song_create_feature(song_df, n_set, n_artist, n_song_min, n_song_artist_max)

    # prepare data
    X, y = prepare_data(artist_song_feature['train'], feature_org, feature_abs)

    pipeline = pipeline.fit(X, y)
    
    return artist_song_feature, pipeline

## Model Initialize

In [ ]:
pipeline = Pipeline([('scale', StandardScaler()), 
                     ('clf', LogisticRegression(solver='lbfgs', max_iter=3000, 
                                                class_weight={False: 1/n_artist, True:(n_artist - 1)/n_artist}))])

In [ ]:
# prepare data create and train pipeline
n_artist = 3
n_song_min = 5
n_set = {'train': 100}
n_song_artist_max = 100

feature_org = ['n_stems', 'unique_stems_ratio', 'stems_per_line', 'tf_idf_score', 'polarity', 'vector_similarity']
feature_abs = ['n_stems_diff', 'n_stems_diff_std', 'unique_stems_ratio_diff', 'unique_stems_ratio_diff_std', 
               'stems_per_line_diff', 'stems_per_line_diff_std', 'tf_idf_score_diff', 'tf_idf_score_diff_std', 
               'polarity_diff', 'polarity_diff_std']

np.random.seed(1)
artist_song_feature, pipeline = select_songs_train_pipeline(song_df, n_set, n_artist, n_song_min, n_song_artist_max, 
                                                            feature_org, feature_abs, pipeline)

Number of songs in train: 3300


In [ ]:
feature_importance_df = pd.DataFrame({'feature': feature_org+feature_abs, 'coefficient':pipeline['clf'].coef_[0]})

px.bar(feature_importance_df.sort_values('coefficient'), x='feature', y='coefficient')

In [97]:
def predict_artist(df, feature_org, feature_abs, pipeline, top_n):
    # prepare data
    X, y = prepare_data(df, feature_org, feature_abs)
    
    # get probability
    proba = pipeline.predict_proba(X)
    # attach to dataframe
    df['probability'] = proba[:, 1]
    df['correct_prediction'] = df['artist_artist'] == df['artist_song']
    
    # get artist song pairs with highest probability
    predict_select = df.sort_values('probability', ascending=False).groupby(['artist_set_id']).head(top_n)\
                       .groupby(['artist_set_id'])['correct_prediction'].max()
    
    # get accuracy
    print('Accuracy: {}'.format(predict_select.mean()))
    
    return predict_select

In [ ]:
artist_predict_df = predict_artist(artist_song_feature['train'], feature_org, feature_abs, pipeline, top_n=1)

Accuracy: 0.94


In [ ]:
artist_predict_df = predict_artist(artist_song_feature['train'], feature_org, feature_abs, pipeline, top_n=2)

Accuracy: 0.99


# Train and Evaluate score

In [ ]:
n_artist_lst = [2, 4, 8, 16, 32, 64, 128]
top_n_lst = [1, 2, 4, 8, 16, 32, 64]
# n_artist_lst = [2, 4, 8, 16]
# top_n_lst = [1, 2, 4, 8]
n_song_min = 5
n_song_artist_max = 128
np.random.seed(2)

n_set = {'train': 100, 'val': 100}

feature_org = ['n_stems', 'unique_stems_ratio', 'stems_per_line', 'tf_idf_score', 'polarity', 'vector_similarity']
feature_abs = ['n_stems_diff', 'n_stems_diff_std', 'unique_stems_ratio_diff', 'unique_stems_ratio_diff_std', 
               'stems_per_line_diff', 'stems_per_line_diff_std', 'tf_idf_score_diff', 'tf_idf_score_diff_std', 
               'polarity_diff', 'polarity_diff_std']

pipeline = Pipeline([('scale', StandardScaler()), 
                     ('clf', LogisticRegression(solver='lbfgs', max_iter=3000, 
                                                class_weight={False: 1/n_artist, True:(n_artist - 1)/n_artist}))])

result_lst = []

for n_artist in n_artist_lst:
    print(datetime.now())
    print('n_artist: {}'.format(n_artist))
    
    artist_song_feature, pipeline = select_songs_train_pipeline(song_df, n_set, n_artist, n_song_min, 
                                                                n_song_artist_max, feature_org, feature_abs, pipeline)
    
    for top_n in [n for n in top_n_lst if n < n_artist]:
        print('top_n: {}'.format(top_n))
        
        predict_select = predict_artist(artist_song_feature['val'], feature_org, feature_abs, pipeline, top_n=top_n)
        
        result_dict = {'n_artist': n_artist, 'top_n': top_n, 'accuracy': predict_select.mean()}
        result_lst.append(result_dict)
        
    print('')
    
result_df = pd.DataFrame(result_lst)

2022-04-26 17:40:47.522814
n_artist: 2
Number of songs in train: 6124
Number of songs in val: 6276
top_n: 1
Accuracy: 1.0

2022-04-26 17:42:23.715412
n_artist: 4
Number of songs in train: 3200
Number of songs in val: 3200
top_n: 1
Accuracy: 0.81
top_n: 2
Accuracy: 0.95

2022-04-26 17:43:39.600071
n_artist: 8
Number of songs in train: 1600
Number of songs in val: 1600
top_n: 1
Accuracy: 0.81
top_n: 2
Accuracy: 0.97
top_n: 4
Accuracy: 0.99

2022-04-26 17:44:46.214880
n_artist: 16
Number of songs in train: 800
Number of songs in val: 800
top_n: 1
Accuracy: 0.69
top_n: 2
Accuracy: 0.83
top_n: 4
Accuracy: 0.89
top_n: 8
Accuracy: 0.97

2022-04-26 17:45:52.709413
n_artist: 32
Number of songs in train: 400
Number of songs in val: 400
top_n: 1
Accuracy: 0.42
top_n: 2
Accuracy: 0.63
top_n: 4
Accuracy: 0.72
top_n: 8
Accuracy: 0.89
top_n: 16
Accuracy: 0.98

2022-04-26 17:46:56.715696
n_artist: 64
Number of songs in train: 200
Number of songs in val: 200
top_n: 1
Accuracy: 0.33
top_n: 2
Accuracy: 0

## Visualize Evaluation result

In [101]:
fig = px.line(result_df, x='n_artist', y='accuracy', color='top_n', 
              title='Accuracy vs number of artist and number of top selections', 
              labels={'n_artist': 'Number of artists per set', 'top_n': 'Top predictions'})\
        .update_traces(mode='lines+markers')
fig.show()

# Next step
- Parameter tuing
- Addition Feature
- PCA
- word2vec (maybe)